In [4]:
import cv2
import numpy as np

cap = cv2.VideoCapture('roadcamera.mp4')

bgs = cv2.createBackgroundSubtractorMOG2()

ww = hh = 80
y1 = 550
offset = 6
carcount = 0
detect = []

def get_center(x, y, w, h):
    w1 = int(w / 2)
    h1 = int(h / 2)
    cx = x + w1
    cy = y + h1
    return cx, cy

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3, 3), 5)
    bgss = bgs.apply(blur)

    dilated = cv2.dilate(bgss, np.ones((5, 5)))
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    cv2.line(frame, (25, y1), (600, y1), (176, 130, 39), 5)

    for (i, c) in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(c)
        valid_contour = h >= hh and w >= ww
        if not valid_contour:
            continue
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        center = get_center(x, y, w, h)
        detect.append(center)
        cv2.circle(frame, center, 4, (0, 0, 255), -1)

        for (x, y) in detect:
            if y1 - offset <= y <= y1 + offset:
                carcount += 1
                cv2.line(frame, (25, y1), (1200, y1), (0, 127, 255), 5)
                detect.remove((x, y))
                print("number of cars:", carcount)

    cv2.putText(frame, 'Vehicle count: ' + str(carcount), (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

    cv2.imshow("dilated", dilated)
    cv2.imshow("bgss", bgss)
    cv2.imshow("frame", frame)

    if cv2.waitKey(10) == 27:
        break

cv2.destroyAllWindows()
cap.release()
